### **i.	Разработка и тестирование синтаксического анализатора для обработки html страниц**

Библиотеки/модули:

- `requests` для удобного выполнения HTTP-запросов
- `BeautifulSoup` для синтаксического анализа HTML 
- `sleep`
- `pandas` для итоговой таблицы

In [2]:
import requests
from bs4 import BeautifulSoup as BS
from time import sleep
import pandas as pd

В качестве веб-ресурса для парсинга был взят [Нож](https://knife.media/), который является российским независимым интернет-изданием (журнал о культуре и обществе). Для извлечения данных была выбрана [новостная страница](https://knife.media/category/news/').

In [3]:
url = 'https://knife.media/category/news/'
r = requests.get(url)
print(r.status_code) # проверим подключение

200


In [4]:
soup = BS(r.text, 'lxml')

Далее просто извлекаю релевантную информацию:

In [5]:
link = soup.find('div', class_='widget-news__wrapper').find('a', class_='widget-news__content-link').get('href')
link

'https://knife.media/cholerae-who/'

In [6]:
title = soup.find('div', class_='widget-news__wrapper').find('a', class_='widget-news__content-link').text
title

'Больше миллиарда людей в мире могут заболеть холерой — ВОЗ'

In [7]:
time = soup.find('div', class_='widget-news__wrapper').find('span', class_='meta__item').text
time

'20:52'

In [8]:
date = soup.find('div', class_='widget-news__wrapper').findAll('span', class_='meta__item')[1].text
date

'24 февраля'

In [9]:
# ????

topics = soup.find('div', class_='widget-news__wrapper').findAll('a', class_='meta__item')[0].text + ', ' + soup.find('div', class_='widget-news__wrapper').findAll('a', class_='meta__item')[1].text
topics

'мир, здоровье'

In [10]:
# ???? - как нормально названия топиков получить?? - чтобы потом в цикл загнать

topics = soup.find('div', class_='widget-news__wrapper').findAll('a', class_='meta__item')
topics

[<a class="meta__item" href="https://knife.media/tag/world/">мир</a>,
 <a class="meta__item" href="https://knife.media/tag/health/">здоровье</a>]

Тут задаю цикл для парсинга:

In [11]:
data = []

for p in range(1, 6):
    print(p)

    url = f'https://knife.media/category/news/page/{p}'
    r = requests.get(url)
    sleep(3)
    soup = BS(r.text, 'lxml')

    titles = soup.findAll('div', class_='widget-news__wrapper')

    for title in titles:
        article_title = title.find('a', class_='widget-news__content-link').text
        link = title.find('a', class_='widget-news__content-link').get('href')
        time = title.find('span', class_='meta__item').text
        date = title.findAll('span', class_='meta__item')[1].text

        data.append([article_title, link, time, date])

1
2
3
4
5


In [12]:
len(data)

100

Выгружаю данные:

In [13]:
header = ['article_title', 'link', 'date', 'time']
df = pd.DataFrame(data, columns=header)
df.to_csv('/home/ermak/semester_2/Automated_systems/automated_systems/news.csv', sep=';', encoding='utf8')